# Ejemplo 009

**Problema:** Queremos obtener, para todos los sitios de la red: 
- Código de sitio
- EMG
- Nombre de sitio
- Latitud
- Longitud
- SiteStatus
- StationType
- Región
- Lista de eNODEB-ID 
- Lista de gNODEB-ID
- RSH TP-Anf/Hue
- RSH MOCN/MORAN

#### Importamos paquetes necesarios

In [2]:
# Importación de paquetes requeridos
import os
import io
import requests
import pandas as pd
import numpy as np

from IPython.display import display, Markdown, Latex

import json

#### Funciones auxiliares

In [3]:
# Función auxiliar para visualizar los datos
def pp(obj):
    print(json.dumps(obj, indent=2))

In [4]:
# La operación queda con la forma siguiente:
# «para cada fila r, convertir el valor de la columna "col" en un tipo numérico (pd.to_numeric),
# ignorando los valores nulos (errors='coerce'), redondeando hacia abajo al entero más cercano
# (np.floor) y almacenándolo como entero de 64 bits (astype('Int64'))»

def pd_to_int(df: pd.DataFrame, col: str) -> int:
    return np.floor(pd.to_numeric(df[col], errors='coerce')).astype('Int64')

#### Parámetros del script

In [5]:
# Parmámetros
iga_api_base_url = 'http://ing-acc-movil01.personal.corp/ingenier@/symfony/public/index.php/api'

#### Test de conectividad

In [6]:
# Llamamos a un endpoint de prueba en la API de IGA para verificar conectividad
x = requests.get(iga_api_base_url + '/public/db-sync-state')
pp(x.json())

{
  "status": "success",
  "data": {
    "Uid": "82635",
    "State": "DB_SYNC_STATE_UPDATING",
    "LastSyncTimestamp": "14-09-2024 15:35:14",
    "SyncState": "OK",
    "ProjectsUpdated": "DataManager, AMBA, Litoral Sur"
  }
}


#### Descarga de reportes

- Debemos descargar tres reportes: Datos Generales de Sitio, Datos de Integración, (RAN Sharing) - Datos generales. 
- Nos interesa obtener datos para todos los sitios de la red
- El endpoint usado es /public/report/generic-report en los tres casos.
- Los datos devueltos están en formato binario, y pueden ser leídos directamente por Pandas como un archivo .xlsx, y convertidos a un objeto DataFrame

In [8]:
# Parámetros en el payload de la request con que llamamos al endpoint de IGA:
payload = {
    # Nombre del reporte (lo completamos después)
    "reportName": None,
    # Sitios seleccionados: ninguno
    "siteNames": [],
    # Parámetros de filtrado: todos los sitios de la red (filtro vacío)
    "filteringParams": {},
    # Seleccionar por nombres de sitio: no
    "selectBySiteNames": False,
    # Seleccionar por parámetros de filtrado: sí
    "selectByFilteringParams": True
}

# Datos de Integración
payload["reportName"] = "datos_integracion"
x = requests.post(iga_api_base_url + '/public/report/generic-report', json=payload)
# En x almacenamos los datos devueltos por el endpoint de IGA.
# Es un archivo xlsx en formato binario

# Usando pandas, con la función read_excel, leemos los datos y los almacenamos
# en memoria como un objeto DataFrame:
with io.BytesIO(x.content) as fh:
    df_integracion_4g_raw = pd.read_excel(fh, "LTE")
    df_integracion_5g_raw = pd.read_excel(fh, "NR")

# Repetimos para los demás reportes:
payload["reportName"] = "datos_generales_sitio"
x = requests.post(iga_api_base_url + '/public/report/generic-report', json=payload)
with io.BytesIO(x.content) as fh:
    df_datos_generales_sitio_raw = pd.read_excel(fh, "Datos generales de sitio")
    
payload["reportName"] = "rsh_datos_generales"
x = requests.post(iga_api_base_url + '/public/report/generic-report', json=payload)
with io.BytesIO(x.content) as fh:
    df_rsh_datos_generales_raw = pd.read_excel(fh, "Datos generales")


display(df_datos_generales_sitio_raw)
display(df_integracion_4g_raw)
display(df_integracion_5g_raw)
display(df_rsh_datos_generales_raw)

,Código,EMG,Nombre,Tipo Estación,Estado Sitio,Latitud,Longitud,Región,Provincia,Departamento,...,Búsq. Altura,Búsq. Radio,Altura Estruc.,Propiet. Cota 0,Propiet. Estruc.,Tipo Estruc.,Plan de Obra,Proyecto,Clutter,Clase Sitio
0,1_UNAGRO,NaN,El Palmar,Macro,CT,-27.166490,-61.690980,LINO,Chaco,12 de Octubre,...,58.0,1000.0,60.0,Mástil con riendas,TECO - Personal,Mástil con riendas convencional,NaN,LINO,Barren,C
1,2_UNAGRO,NaN,El Palmar,Macro,CT,-27.199777,-61.845519,LINO,Chaco,12 de Octubre,...,58.0,1000.0,60.0,Mástil con riendas,TECO - Personal,Mástil con riendas convencional,NaN,LINO,Grass-Agriculture,C
2,A01-E133,XRCESE,Río IV E Sarmiento,EasyM,OA,-33.142181,-64.348739,MEDI,Córdoba,Río Cuarto,...,12.0,50.0,NaN,NaN,NaN,Poste (con o sin luminaria),NaN,MEDI,Barren,D
3,A01-E134,XRCEAL,Río IV LN Alem,EasyM,OA,-33.141588,-64.338766,MEDI,Córdoba,Río Cuarto,...,12.0,50.0,NaN,NaN,NaN,Poste (con o sin luminaria),NaN,MEDI,Suburban,D
4,A01-E135,XRCEMO,Río IV E Mosconi,EasyM,OA,-33.149821,-64.354526,MEDI,Córdoba,Río Cuarto,...,12.0,50.0,NaN,NaN,NaN,Poste (con o sin luminaria),NaN,MEDI,Suburban,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19750,SPU996,XXYYZZ,AMBA BERAZATEGUI 2020 - 018,Macro,BS,-34.777053,-58.259333,AMBA,Buenos Aires,Berazategui,...,20.0,100.0,20.0,NaN,0,NaN,NaN,AMBA,Suburban,C
19751,SPU997,XXYYZZ,AMBA BERAZATEGUI 2020 - 019,Macro,BS,-34.766352,-58.238844,AMBA,Buenos Aires,Berazategui,...,20.0,100.0,20.0,NaN,0,NaN,NaN,AMBA,Suburban,C
19752,SPU998,SBB12O,Berazategui Barrio 12 de Octubre,Macro,BS,-34.767692,-58.229362,AMBA,Buenos Aires,Berazategui,...,20.0,100.0,25.0,NaN,NaN,NaN,NaN,AMBA,Suburban,C
19753,SPU999,XXYYZZ,AMBA BERAZATEGUI 2020 - 021,Macro,BS,-34.760799,-58.222894,AMBA,Buenos Aires,Berazategui,...,20.0,100.0,20.0,NaN,0,NaN,NaN,AMBA,Suburban,C


,Proyecto,Código,Sector,Portadora,MME,CGI,Estado,Vendor,E-NODEB Id,TAC,Physical Cell Id,Logical Cell Id,RACH Root Seq.,PA Power dBm,Ref. Signal Boosting dB,EARFCN DL,EARFCN UL,PRACH CS,Base Station Type,E-UTRAN Cell Id
0,LINO,1_UNAGRO,HELPLMB11,LTE FDD_700MHz_9312_NB_IoT,MME[1_2CLI-1_2BEL-1_2MUN],722-34-,NaN,NOKIA,NaN,31696,-1,NaN,NaN,38.0,3.00,9312.0,27312.0,NaN,FlexiSystemModule,NaN
1,LINO,1_UNAGRO,HELPLMB12,LTE FDD_700MHz_9312_NB_IoT,MME[1_2CLI-1_2BEL-1_2MUN],722-34-,NaN,NOKIA,NaN,31696,-1,NaN,NaN,38.0,3.00,9312.0,27312.0,NaN,FlexiSystemModule,NaN
2,LINO,1_UNAGRO,HELPLMB13,LTE FDD_700MHz_9312_NB_IoT,MME[1_2CLI-1_2BEL-1_2MUN],722-34-,NaN,NOKIA,NaN,31696,-1,NaN,NaN,38.0,3.00,9312.0,27312.0,NaN,FlexiSystemModule,NaN
3,LINO,1_UNAGRO,HELPLMN11,LTE FDD_700MHz_9360,MME[1_2CLI-1_2BEL-1_2MUN],722-34-,NaN,NOKIA,NaN,11696,-1,NaN,638.0,46.0,0.00,9360.0,27360.0,14.0,FlexiSystemModule,NaN
4,LINO,1_UNAGRO,HELPLMN12,LTE FDD_700MHz_9360,MME[1_2CLI-1_2BEL-1_2MUN],722-34-,NaN,NOKIA,NaN,11696,-1,NaN,500.0,46.0,0.00,9360.0,27360.0,14.0,FlexiSystemModule,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178110,AMBA,Yacht Club,NDELTPN12,LTE FDD_700MHz_9360,MME[1_2CLI-1_2BEL-1_2MUN],722-34-,No Operativo,Huawei,NaN,15102,160,NaN,310.0,48.4,1.77,9360.0,27360.0,12.0,BBU 3900,NaN
178111,AMBA,Yacht Club,NDELTPN13,LTE FDD_700MHz_9360,MME[1_2CLI-1_2BEL-1_2MUN],722-34-,No Operativo,Huawei,NaN,15102,161,NaN,330.0,48.4,1.77,9360.0,27360.0,12.0,BBU 3900,NaN
178112,AMBA,Yacht Club,NDELTPO11,LTE FDD_850MHz_2450,MME[1_2CLI-1_2BEL-1_2MUN],722-34-,No Operativo,Huawei,NaN,15102,162,NaN,300.0,48.4,1.77,2450.0,20450.0,12.0,BBU 3900,NaN
178113,AMBA,Yacht Club,NDELTPO12,LTE FDD_850MHz_2450,MME[1_2CLI-1_2BEL-1_2MUN],722-34-,No Operativo,Huawei,NaN,15102,160,NaN,310.0,48.4,1.77,2450.0,20450.0,12.0,BBU 3900,NaN


,Proyecto,Código,Sector,Portadora,MME,CGI,Estado,Vendor,G-NODEB Id,TAC,Physical Cell Id,Logical Cell Id,RACH Root Seq.,PA Power dBm,EARFCN DL,Base Station Type,NR Cell Id
0,MEDI,A01-S006,XLCARLR11,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32931938304,No Operativo,HUAWEI,2010006.0,10192,357,0.0,6,50.0,534000,BBU 3900,3.293194e+10
1,MEDI,A01-S006,XLCARLR12,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32931938305,No Operativo,HUAWEI,2010006.0,10192,360,1.0,12,50.0,534000,BBU 3900,3.293194e+10
2,MEDI,A01-S006,XLCARLR13,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32931938306,No Operativo,HUAWEI,2010006.0,10192,363,2.0,18,50.0,534000,BBU 3900,3.293194e+10
3,MEDI,A01-S018,XMAKENR11,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32932134912,No Operativo,HUAWEI,2010018.0,10192,9,0.0,6,50.0,534000,BBU 3900,3.293213e+10
4,MEDI,A01-S018,XMAKENR12,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-32932134913,No Operativo,HUAWEI,2010018.0,10192,12,1.0,12,50.0,534000,BBU 3900,3.293213e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3890,AMBA,SPU674,SPPMTPR12,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-41020194817,No Operativo,Huawei,2503674.0,15118,478,1.0,54,50.0,534000,BBU 5900,4.102019e+10
3891,AMBA,SPU674,SPPMTPR13,NR_Band_n7_534000,MME[1_2CLI-1_2BEL-1_2MUN],722-34-41020194818,No Operativo,Huawei,2503674.0,15118,479,2.0,66,50.0,534000,BBU 5900,4.102019e+10
3892,AMBA,SPU816,SCPESES11,NR_Band_n78_630000,NaN,722-34-,No Operativo,NaN,NaN,15117,-1,350.0,44,50.0,630000,BBU 5900,NaN
3893,AMBA,SPU816,SCPESES12,NR_Band_n78_630000,NaN,722-34-,No Operativo,NaN,NaN,15117,-1,351.0,0,50.0,630000,BBU 5900,NaN


,Código Sitio (TP),Tipo Estación,Estado Sitio,Latitud,Longitud,Nombre Celda (TP),Portadora,Modalidad RSH,MOCN/MORAN,Sector ID,Operador,Código Sitio (Oper),Nodo (Oper),Nombre Celda (Oper),Banda,Notas,Proyecto RSH,Vendor 2G,Vendor 3G,Vendor 4G
0,A01-S002,Macro,OA,-34.722105,-63.505801,XHBOUCN11,LTE FDD_700MHz_9360,TP Anfitrión,MOCN,1,Movistar,XS145,MXS145,MXS145A,B28,NaN,Etapa 5,H,H,H
1,A01-S002,Macro,OA,-34.722105,-63.505801,XHBOUCN12,LTE FDD_700MHz_9360,TP Anfitrión,MOCN,2,Movistar,XS145,MXS145,MXS145B,B28,NaN,Etapa 5,H,H,H
2,A01-S002,Macro,OA,-34.722105,-63.505801,XHBOUCN13,LTE FDD_700MHz_9360,TP Anfitrión,MOCN,3,Movistar,XS145,MXS145,MXS145C,B28,NaN,Etapa 5,H,H,H
3,A01-S003,Macro,OA,-34.517722,-63.946917,XSMAGDN11,LTE FDD_700MHz_9360,TP Anfitrión,MOCN,1,Movistar,XS116,MXS116,MXS116A,B28,NaN,Etapa 4,H,H,H
4,A01-S003,Macro,OA,-34.517722,-63.946917,XSMAGDN12,LTE FDD_700MHz_9360,TP Anfitrión,MOCN,2,Movistar,XS116,MXS116,MXS116B,B28,NaN,Etapa 4,H,H,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4951,SPU2280,Macro,CT,-34.737090,-58.330140,SQURSHL12,LTE FDD_AWS_2225,TP Huésped,MORAN,2,Movistar,KSM03,LKSM03,LKSM03B,B4,CSFB pedido a 3G,Centros de Emergencia - Hospitales COVID-19,NaN,NaN,NaN
4952,SPU2280,Macro,CT,-34.737090,-58.330140,SQURSHL13,LTE FDD_AWS_2225,TP Huésped,MORAN,3,Movistar,KSM03,LKSM03,LKSM03C,B4,CSFB pedido a 3G,Centros de Emergencia - Hospitales COVID-19,NaN,NaN,NaN
4953,SPU2280,Macro,CT,-34.737090,-58.330140,SQURSHN11,LTE FDD_700MHz_9360,TP Huésped,MORAN,1,Movistar,KSM03,MKSM03,MKSM03A,B28,CSFB pedido a 3G,Centros de Emergencia - Hospitales COVID-19,NaN,NaN,NaN
4954,SPU2280,Macro,CT,-34.737090,-58.330140,SQURSHN12,LTE FDD_700MHz_9360,TP Huésped,MORAN,2,Movistar,KSM03,MKSM03,MKSM03B,B28,CSFB pedido a 3G,Centros de Emergencia - Hospitales COVID-19,NaN,NaN,NaN


#### Procesamiento de los datos

- De los datos de Integración, obtenemos para cada sitio los E-NODEB Ids y G-NODEB Ids
- De los datos de RSH, obtenemos para cada sitio las modalidades de RSH y MOCN/MORAN
- Unimos las tres tablas usando el campo Código de sitio

In [44]:
df_1 = (
    df_datos_generales_sitio_raw
    [['Código', 'EMG', 'Nombre', 'Tipo Estación', 'Estado Sitio', 'Latitud', 'Longitud', 'Región']]
).set_index('Código')
display(df_1)

df_2 = (
    df_integracion_4g_raw[df_integracion_4g_raw['E-NODEB Id'].notnull()]
    [['Código', 'E-NODEB Id']]
).drop_duplicates().groupby('Código')['E-NODEB Id'].apply(list).reset_index().set_index('Código')
display(df_2)

df_3 = (
    df_integracion_5g_raw[df_integracion_5g_raw['G-NODEB Id'].notnull()]
    [['Código', 'G-NODEB Id']]
).drop_duplicates().groupby('Código')['G-NODEB Id'].apply(list).reset_index().set_index('Código')
display(df_3)

df_4 = (
    df_rsh_datos_generales_raw
    [['Código Sitio (TP)', 'Modalidad RSH']]
).drop_duplicates().groupby('Código Sitio (TP)')['Modalidad RSH'].apply(list).reset_index().set_index('Código Sitio (TP)')
display(df_4)

df_5 = (
    df_rsh_datos_generales_raw
    [['Código Sitio (TP)', 'MOCN/MORAN']]
).drop_duplicates().groupby('Código Sitio (TP)')['MOCN/MORAN'].apply(list).reset_index().set_index('Código Sitio (TP)')
display(df_5)

,EMG,Nombre,Tipo Estación,Estado Sitio,Latitud,Longitud,Región
Código,,,,,,,
1_UNAGRO,NaN,El Palmar,Macro,CT,-27.166490,-61.690980,LINO
2_UNAGRO,NaN,El Palmar,Macro,CT,-27.199777,-61.845519,LINO
A01-E133,XRCESE,Río IV E Sarmiento,EasyM,OA,-33.142181,-64.348739,MEDI
A01-E134,XRCEAL,Río IV LN Alem,EasyM,OA,-33.141588,-64.338766,MEDI
A01-E135,XRCEMO,Río IV E Mosconi,EasyM,OA,-33.149821,-64.354526,MEDI
...,...,...,...,...,...,...,...
SPU996,XXYYZZ,AMBA BERAZATEGUI 2020 - 018,Macro,BS,-34.777053,-58.259333,AMBA
SPU997,XXYYZZ,AMBA BERAZATEGUI 2020 - 019,Macro,BS,-34.766352,-58.238844,AMBA
SPU998,SBB12O,Berazategui Barrio 12 de Octubre,Macro,BS,-34.767692,-58.229362,AMBA


,E-NODEB Id
Código,
A01-E133,[10133.0]
A01-E134,[10133.0]
A01-E135,[10133.0]
A01-E136,[10133.0]
A01-E138,[10138.0]
...,...
SPU995,[503995.0]
SPU996,[503996.0]
SPU997,[503997.0]


,G-NODEB Id
Código,
A01-S006,[2010006.0]
A01-S018,[2010018.0]
A01-S021,[2010021.0]
A01-S035,[2010035.0]
A01-S036,[2010036.0]
...,...
SPU594,[2503594.0]
SPU596,[2503596.0]
SPU663,[2503663.0]


,Modalidad RSH
Código Sitio (TP),
A01-S002,[TP Anfitrión]
A01-S003,[TP Anfitrión]
A01-S004,[TP Anfitrión]
A01-S008,[TP Anfitrión]
A01-S011,[TP Anfitrión]
...,...
SI_029,[TP Anfitrión]
SI_030,[TP Huésped]
SI_040,[TP Anfitrión]


,MOCN/MORAN
Código Sitio (TP),
A01-S002,[MOCN]
A01-S003,[MOCN]
A01-S004,[MOCN]
A01-S008,[MOCN]
A01-S011,[MOCN]
...,...
SI_029,[MORAN]
SI_030,[MORAN]
SI_040,[MOCN]


In [45]:
df = df_1.join(df_2, how='left').join(df_3, how='left').join(df_4, how='left').join(df_5, how='left')
display(df)

,EMG,Nombre,Tipo Estación,Estado Sitio,Latitud,Longitud,Región,E-NODEB Id,G-NODEB Id,Modalidad RSH,MOCN/MORAN
Código,,,,,,,,,,,
1_UNAGRO,NaN,El Palmar,Macro,CT,-27.166490,-61.690980,LINO,NaN,NaN,NaN,NaN
2_UNAGRO,NaN,El Palmar,Macro,CT,-27.199777,-61.845519,LINO,NaN,NaN,NaN,NaN
A01-E133,XRCESE,Río IV E Sarmiento,EasyM,OA,-33.142181,-64.348739,MEDI,[10133.0],NaN,NaN,NaN
A01-E134,XRCEAL,Río IV LN Alem,EasyM,OA,-33.141588,-64.338766,MEDI,[10133.0],NaN,NaN,NaN
A01-E135,XRCEMO,Río IV E Mosconi,EasyM,OA,-33.149821,-64.354526,MEDI,[10133.0],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
SPU996,XXYYZZ,AMBA BERAZATEGUI 2020 - 018,Macro,BS,-34.777053,-58.259333,AMBA,[503996.0],NaN,NaN,NaN
SPU997,XXYYZZ,AMBA BERAZATEGUI 2020 - 019,Macro,BS,-34.766352,-58.238844,AMBA,[503997.0],NaN,NaN,NaN
SPU998,SBB12O,Berazategui Barrio 12 de Octubre,Macro,BS,-34.767692,-58.229362,AMBA,[503998.0],NaN,NaN,NaN


In [57]:
# Algunos ejemplos:

print(df[df.index == 'A18-S141'].to_json(force_ascii=False, indent=2, orient='records'))
print(df[df.index == 'A38-A683'].to_json(force_ascii=False, indent=2, orient='records'))

[
  {
    "EMG":"IPFFCC",
    "Nombre":"Posadas Estacion FFCC",
    "Tipo Estación":"Macro",
    "Estado Sitio":"OA",
    "Latitud":-27.422534,
    "Longitud":-55.881409,
    "Región":"LINO",
    "E-NODEB Id":[
      180141.0,
      183141.0,
      187141.0,
      182141.0,
      186141.0
    ],
    "G-NODEB Id":null,
    "Modalidad RSH":null,
    "MOCN\/MORAN":null
  }
]
[
  {
    "EMG":"BBBRSH",
    "Nombre":"BBlanca Villa Bordeu RSH",
    "Tipo Estación":"Macro",
    "Estado Sitio":"OA",
    "Latitud":-38.687398,
    "Longitud":-62.341009,
    "Región":"SUR",
    "E-NODEB Id":[
      900003.0
    ],
    "G-NODEB Id":null,
    "Modalidad RSH":[
      "TP Huésped"
    ],
    "MOCN\/MORAN":[
      "MOCN",
      "MORAN"
    ]
  }
]
